In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## Discriminator

In [18]:
def define_discriminator(in_shape=(28, 28, 1)):
  init = keras.initializers.RandomNormal(stddev=0.02)
  model = keras.Sequential()
  model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init, input_shape=in_shape))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(0.2))
  model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation="sigmoid"))
  model.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5),
                metrics=['accuracy'])
  return model

## Generator

In [19]:
def define_generator(latent_dim):
  init = keras.initializers.RandomNormal(stddev=0.02)
  model = keras.Sequential()
  n_nodes = 128 * 7 * 7
  model.add(layers.Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
  model.add(layers.LeakyReLU(0.2))
  model.add(layers.Reshape((7, 7, 128)))
  model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(0.2))
  model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(0.2))
  model.add(layers.Conv2D(1, (7, 7), activation='tanh', padding='same', kernel_initializer=init))
  return model

## GAN

In [20]:
def define_gan(generator, discriminator):
  discriminator.trainable = False
  model = keras.Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5))
  return model

## Load the dataset

In [21]:
def load_real_samples():
  (x_train, y_train), (_, _) = keras.datasets.mnist.load_data()
  x = np.expand_dims(x_train, axis=-1)
  selected_ix = y_train == 8
  x = x[selected_ix]
  x = x.astype('float32')
  x = (x - 127.5) / 127.5
  return x

## Generating Functions

In [22]:
def generate_real_samples(dataset, n_samples):
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  x = dataset[ix]
  y = np.ones((n_samples, 1))
  return x, y

def generate_latent_points(latent_dim, n_samples):
  x_input = np.random.randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  return z_input

def generate_fake_samples(generator, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  x = generator.predict(x_input)
  y = np.zeros((n_samples, 1))
  return x, y

def summarize_performance(step, g_model, latent_dim, n_samples=100):
  x, _ = generate_fake_samples(g_model, latent_dim, n_samples)
  x = (x + 1) / 2.0
  for i in range(10 * 10):
    plt.subplot(10, 10, 1 + i)
    plt.axis('off')
    plt.imshow(x[i, :, :, 0], cmap='gray_r')
  plt.savefig('results_baseline/generated_plot_%03d.png' % (step+1))
  plt.close() # save the generator model
  g_model.save('results_baseline/model_%03d.h5' % (step+1))

def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
  # plot loss
  plt.subplot(2, 1, 1) 
  plt.plot(d1_hist, label='d-real') 
  plt.plot(d2_hist, label='d-fake') 
  plt.plot(g_hist, label='gen') 
  plt.legend() 
  # plot discriminator accuracy 
  plt.subplot(2, 1, 2) 
  plt.plot(a1_hist, label='acc-real') 
  plt.plot(a2_hist, label='acc-fake') 
  plt.legend() 
  # save plot to file 
  plt.savefig('results_baseline/plot_line_plot_loss.png')
  plt.close()

## Train the GAN

In [23]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  n_steps = bat_per_epo * n_epochs
  half_batch = int(n_batch / 2)
  d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
  for i in range(n_steps):
    x_real, y_real = generate_real_samples(dataset, half_batch)
    d_loss1, d_acc1 = d_model.train_on_batch(x_real, y_real)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    d_loss2, d_acc2 = d_model.train_on_batch(x_fake, y_fake)
    x_gan = generate_latent_points(latent_dim, n_batch)
    y_gan = np.ones((n_batch, 1))
    g_loss = gan_model.train_on_batch(x_gan, y_gan)
    print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' 
          % (i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
    d1_hist.append(d_loss1)
    d2_hist.append(d_loss2)
    g_hist.append(g_loss)
    a1_hist.append(d_acc1)
    a2_hist.append(d_acc2)
    if (i+1) % bat_per_epo == 0: 
      summarize_performance(i, g_model, latent_dim)
  plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

In [24]:
os.makedirs('results_baseline' , exist_ok=True)
latent_dim = 100
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)
dataset = load_real_samples()
print(dataset.shape)
train(generator, discriminator, gan_model, dataset, latent_dim)

(5851, 28, 28, 1)
2/2 [==============================] - 0s 6ms/step
>1, d1=1.046, d2=1.340 g=0.719, a1=26, a2=3
2/2 [==============================] - 0s 5ms/step
>2, d1=0.000, d2=5.726 g=0.665, a1=100, a2=0
2/2 [==============================] - 0s 5ms/step
>3, d1=0.000, d2=0.001 g=0.691, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>4, d1=0.002, d2=0.000 g=0.688, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>5, d1=0.003, d2=0.000 g=0.679, a1=100, a2=100
2/2 [==============================] - 0s 4ms/step
>6, d1=0.002, d2=0.001 g=0.679, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>7, d1=0.002, d2=0.001 g=0.677, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>8, d1=0.003, d2=0.001 g=0.674, a1=100, a2=100
2/2 [==============================] - 0s 4ms/step
>9, d1=0.004, d2=0.001 g=0.674, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>10, d1=0.002, d2=0.001 g=0.674, a1=100, a2=100
2/2 [=

2/2 [==============================] - 0s 8ms/step
>46, d1=0.001, d2=0.001 g=0.685, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>47, d1=0.002, d2=0.001 g=0.673, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>48, d1=0.002, d2=0.001 g=0.651, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>49, d1=0.003, d2=0.003 g=0.647, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>50, d1=0.001, d2=0.001 g=0.644, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>51, d1=0.001, d2=0.001 g=0.638, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>52, d1=0.001, d2=0.000 g=0.631, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>53, d1=0.001, d2=0.001 g=0.625, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>54, d1=0.001, d2=0.001 g=0.620, a1=100, a2=100
2/2 [==============================] - 0s 7ms/step
>55, d1=0.000, d2=0.001 g=0.613, a1=100, a2=100
2/2 [=====

2/2 [==============================] - 0s 6ms/step
>91, d1=0.000, d2=0.000 g=0.585, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>92, d1=0.000, d2=0.000 g=0.586, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>93, d1=0.000, d2=0.000 g=0.589, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>94, d1=0.000, d2=0.000 g=0.590, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>95, d1=0.000, d2=0.000 g=0.587, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>96, d1=0.000, d2=0.000 g=0.589, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>97, d1=0.000, d2=0.000 g=0.591, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>98, d1=0.000, d2=0.000 g=0.592, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>99, d1=0.000, d2=0.000 g=0.594, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>100, d1=0.000, d2=0.000 g=0.600, a1=100, a2=100
2/2 [====

2/2 [==============================] - 0s 5ms/step
>136, d1=0.000, d2=0.000 g=0.848, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>137, d1=0.000, d2=0.000 g=0.844, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>138, d1=0.000, d2=0.000 g=0.848, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>139, d1=0.000, d2=0.000 g=0.859, a1=100, a2=100
2/2 [==============================] - 0s 7ms/step
>140, d1=0.000, d2=0.000 g=0.867, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>141, d1=0.000, d2=0.000 g=0.870, a1=100, a2=100
2/2 [==============================] - 0s 10ms/step
>142, d1=0.000, d2=0.000 g=0.871, a1=100, a2=100
2/2 [==============================] - 0s 7ms/step
>143, d1=0.000, d2=0.000 g=0.863, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>144, d1=0.000, d2=0.000 g=0.853, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>145, d1=0.000, d2=0.000 g=0.849, a1=100, a2=100

2/2 [==============================] - 0s 6ms/step
>181, d1=0.000, d2=0.000 g=0.640, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>182, d1=0.000, d2=0.000 g=0.641, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>183, d1=0.000, d2=0.000 g=0.647, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>184, d1=0.000, d2=0.000 g=0.651, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>185, d1=0.000, d2=0.000 g=0.651, a1=100, a2=100
2/2 [==============================] - 0s 11ms/step
>186, d1=0.000, d2=0.000 g=0.659, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>187, d1=0.000, d2=0.000 g=0.654, a1=100, a2=100
2/2 [==============================] - 0s 4ms/step
>188, d1=0.000, d2=0.000 g=0.658, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>189, d1=0.000, d2=0.000 g=0.662, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>190, d1=0.000, d2=0.000 g=0.666, a1=100, a2=100

2/2 [==============================] - 0s 6ms/step
>226, d1=0.000, d2=0.001 g=0.001, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>227, d1=0.000, d2=0.001 g=0.002, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>228, d1=0.000, d2=0.000 g=0.003, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>229, d1=0.000, d2=0.000 g=0.004, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>230, d1=0.000, d2=0.000 g=0.006, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>231, d1=0.000, d2=0.000 g=0.007, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>232, d1=0.000, d2=0.000 g=0.009, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>233, d1=0.000, d2=0.000 g=0.011, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>234, d1=0.000, d2=0.000 g=0.013, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>235, d1=0.000, d2=0.000 g=0.016, a1=100, a2=100


2/2 [==============================] - 0s 7ms/step
>271, d1=0.000, d2=0.000 g=0.178, a1=100, a2=100
2/2 [==============================] - 0s 10ms/step
>272, d1=0.000, d2=0.000 g=0.182, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>273, d1=0.000, d2=0.000 g=0.187, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>274, d1=0.000, d2=0.000 g=0.192, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>275, d1=0.000, d2=0.000 g=0.196, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>276, d1=0.000, d2=0.000 g=0.201, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>277, d1=0.000, d2=0.000 g=0.206, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>278, d1=0.000, d2=0.000 g=0.210, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>279, d1=0.000, d2=0.000 g=0.215, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>280, d1=0.000, d2=0.000 g=0.219, a1=100, a2=100

2/2 [==============================] - 0s 6ms/step
>316, d1=0.000, d2=0.000 g=0.340, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>317, d1=0.000, d2=0.000 g=0.342, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>318, d1=0.000, d2=0.000 g=0.345, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>319, d1=0.000, d2=0.000 g=0.348, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>320, d1=0.000, d2=0.000 g=0.350, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>321, d1=0.000, d2=0.000 g=0.353, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>322, d1=0.000, d2=0.000 g=0.355, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>323, d1=0.000, d2=0.000 g=0.358, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>324, d1=0.000, d2=0.000 g=0.360, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>325, d1=0.000, d2=0.000 g=0.363, a1=100, a2=100


2/2 [==============================] - 0s 6ms/step
>361, d1=0.000, d2=0.000 g=0.434, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>362, d1=0.000, d2=0.000 g=0.436, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>363, d1=0.000, d2=0.000 g=0.437, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>364, d1=0.000, d2=0.000 g=0.439, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>365, d1=0.000, d2=0.000 g=0.441, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>366, d1=0.000, d2=0.000 g=0.442, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>367, d1=0.000, d2=0.000 g=0.444, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>368, d1=0.000, d2=0.000 g=0.446, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>369, d1=0.000, d2=0.000 g=0.447, a1=100, a2=100
2/2 [==============================] - 0s 5ms/step
>370, d1=0.000, d2=0.000 g=0.449, a1=100, a2=100


2/2 [==============================] - 0s 7ms/step
>406, d1=0.000, d2=0.000 g=0.492, a1=100, a2=100
2/2 [==============================] - 0s 7ms/step
>407, d1=0.000, d2=0.000 g=0.494, a1=100, a2=100
2/2 [==============================] - 0s 8ms/step
>408, d1=0.000, d2=0.000 g=0.496, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>409, d1=0.000, d2=0.000 g=0.497, a1=100, a2=100
2/2 [==============================] - 0s 7ms/step
>410, d1=0.000, d2=0.000 g=0.498, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>411, d1=0.000, d2=0.000 g=0.499, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>412, d1=0.000, d2=0.000 g=0.500, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>413, d1=0.000, d2=0.000 g=0.502, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>414, d1=0.000, d2=0.000 g=0.502, a1=100, a2=100
2/2 [==============================] - 0s 6ms/step
>415, d1=0.000, d2=0.000 g=0.503, a1=100, a2=100
